In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from library import datasets, models
from flax import linen as nn
from jax import random, numpy as jnp
import optax
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'notebook_connected'

In [3]:
MODEL_LATENT_DIM = 4
TRUE_LATENT_DIM = 2
AMBIENT_DIM = 2

In [4]:
# taylor_pair_dis = datasets.point_dataset.PointPairDistribution.random_taylor(
#     latent_dim=TRUE_LATENT_DIM,
#     dis_A_dim=AMBIENT_DIM,
#     dis_B_dim=AMBIENT_DIM,
#     latent_range=1,
#     max_order=2,
#     coeff_range=1,
#     noise_std_A=0.025,
#     noise_std_B=0,
#     key=random.PRNGKey(2))

# dataset = taylor_pair_dis.generate_dataset(1000)
dataset = datasets.utils.make_blobs(n_samples=1000, min_sigma=0, max_sigma=0.1)

In [5]:
A = dataset.get_tensors()

In [6]:
discriminator = nn.Sequential([
    nn.Dense(8),
    nn.relu,
    nn.Dense(16),
    nn.relu,
    nn.Dense(16),
    nn.relu,
    nn.Dense(8),
    nn.relu,
    nn.Dense(1),
])

generator = nn.Sequential([
    nn.Dense(8),
    nn.relu,
    nn.Dense(8),
    nn.relu,
    nn.Dense(4),
    nn.relu,
    nn.Dense(AMBIENT_DIM),
])

model = models.vanilla_gan.VanillaGAN(
    generator=generator,
    discriminator=discriminator,
    latent_shape=(MODEL_LATENT_DIM,),
    ambient_shape=(AMBIENT_DIM,)
)

print(model)
model.initialize(loss_fn=optax.sigmoid_binary_cross_entropy)

generator:
	Model Structure:
		Sequential(
		    # attributes
		    layers = [Dense(
		        # attributes
		        features = 8
		        use_bias = True
		        dtype = None
		        param_dtype = float32
		        precision = None
		        kernel_init = init
		        bias_init = zeros
		    ), <jax._src.custom_derivatives.custom_jvp object at 0x7f1c4a7994b0>, Dense(
		        # attributes
		        features = 8
		        use_bias = True
		        dtype = None
		        param_dtype = float32
		        precision = None
		        kernel_init = init
		        bias_init = zeros
		    ), <jax._src.custom_derivatives.custom_jvp object at 0x7f1c4a7994b0>, Dense(
		        # attributes
		        features = 4
		        use_bias = True
		        dtype = None
		        param_dtype = float32
		        precision = None
		        kernel_init = init
		        bias_init = zeros
		    ), <jax._src.custom_derivatives.custom_jvp object at 0x7f1c4a7994b0>, Dense(
		        # attributes
		        

In [7]:
gan_distribution = model.create_distribution()

In [8]:
true_samples = A

In [9]:
fake_samples = gan_distribution.draw_samples(random.uniform(random.PRNGKey(0), (1000, MODEL_LATENT_DIM), minval=-1, maxval=1))
# fake_samples = gan_distribution.draw_samples(random.normal(random.PRNGKey(0), (1000, LATENT_DIM)))

In [10]:
print(true_samples.shape, fake_samples.shape)

(1000, 2) (1000, 2)


In [11]:
df = {'x': jnp.concatenate((true_samples[:, 0], fake_samples[:, 0]), axis=0),
      'y': jnp.concatenate((true_samples[:, 1], fake_samples[:, 1]), axis=0),
      'labels': jnp.concatenate((jnp.ones((true_samples.shape[0],)), jnp.zeros((fake_samples.shape[0],))), axis=0)}

In [12]:
px.scatter(df, x='x', y='y', color='labels')

In [13]:
model.train(datasets.base.TensorDataset(A), optax.adam(learning_rate=1e-3), print_every=5, batch_size=1000, num_epochs=500)

iteration 0; gen_loss:  7.02e-01; dis_loss:  6.92e-01; gen_grads_magnitude:  9.95e-03; dis_grads_magnitude:  2.97e-03: : 1it [00:05,  5.48s/it]
iteration 0; gen_loss:  7.03e-01; dis_loss:  6.91e-01; gen_grads_magnitude:  1.08e-02; dis_grads_magnitude:  3.36e-03: : 1it [00:00,  7.02it/s]
iteration 0; gen_loss:  7.03e-01; dis_loss:  6.91e-01; gen_grads_magnitude:  1.01e-02; dis_grads_magnitude:  3.66e-03: : 1it [00:00,  6.56it/s]
iteration 0; gen_loss:  7.03e-01; dis_loss:  6.91e-01; gen_grads_magnitude:  1.01e-02; dis_grads_magnitude:  3.89e-03: : 1it [00:00,  6.19it/s]
iteration 0; gen_loss:  7.04e-01; dis_loss:  6.91e-01; gen_grads_magnitude:  1.01e-02; dis_grads_magnitude:  4.27e-03: : 1it [00:00,  6.65it/s]
iteration 0; gen_loss:  7.03e-01; dis_loss:  6.91e-01; gen_grads_magnitude:  9.93e-03; dis_grads_magnitude:  4.67e-03: : 1it [00:00,  6.60it/s]
iteration 0; gen_loss:  7.04e-01; dis_loss:  6.91e-01; gen_grads_magnitude:  1.00e-02; dis_grads_magnitude:  4.65e-03: : 1it [00:00,  6.

In [14]:
gan_distribution = model.create_distribution()

In [15]:
true_samples = A

In [16]:
fake_samples = gan_distribution.draw_samples(random.uniform(random.PRNGKey(0), (1000, MODEL_LATENT_DIM), minval=-1, maxval=1))
# fake_samples = gan_distribution.draw_samples(random.normal(random.PRNGKey(0), (1000, LATENT_DIM)))

In [17]:
print(true_samples.shape, fake_samples.shape)

(1000, 2) (1000, 2)


In [18]:
df = {'x': jnp.concatenate((true_samples[:, 0], fake_samples[:, 0]), axis=0),
      'y': jnp.concatenate((true_samples[:, 1], fake_samples[:, 1]), axis=0),
      'labels': jnp.concatenate((jnp.ones((true_samples.shape[0],)), jnp.zeros((fake_samples.shape[0],))), axis=0)}

In [19]:
px.scatter(df, x='x', y='y', color='labels')